# Theano

In [1]:
# data import
import scipy.io
ratdata = scipy.io.loadmat('chrono_B069_rawdata.mat')

In [2]:
# trial index starts from 0 

def trialdata(ratdata,trial):
    if ratdata['rawdata']['pokedR'][0][trial] > 0 :
        rat_choice = 1 # "R"
    else : 
        rat_choice = -1 # "L"
        
    RClickTimes = np.require(ratdata['rawdata']['rightbups'][0][trial][0],requirements='A')
    LClickTimes = np.require(ratdata['rawdata']['leftbups'][0][trial][0],requirements='A')
    T1 = ratdata['rawdata']['T'][0][trial][0][0]
    
    if np.shape(LClickTimes)[0] == 1:
        LClickTimes = [0, LClickTimes]
    if np.shape(RClickTimes)[0] == 1:
        RClickTimes = [0, RClickTimes]
    return  RClickTimes, LClickTimes, T1, rat_choice

In [3]:
import numpy as np
import theano
import theano.tensor as T

In [4]:
# trial index starts from 0 
RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, 0) 
print LeftClickTimes
print maxT
print rat_choice

[ 0.       0.19235  0.34361]
0.350959
-1


In [29]:
# Global variables 
epsilon = 10.0**(-10) 
dx = 0.25
dt = 0.02

# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05;

# Theano shared_variables 
sigma2_a = theano.shared(sigma_a, name="sigma_a")
sigma2_s = theano.shared(sigma_s, name="sigma_s")
sigma2_i = theano.shared(sigma_i, name="sigma_i")
lam = theano.shared(lam, name="lambda")
B = theano.shared(B, name="B")
bias = theano.shared(bias, name="bias")
phi = theano.shared(phi, name="phi")
tau_phi = theano.shared(tau_phi, name="tau_phi")
lapse = theano.shared(lapse, name="lapse")

params = [sigma2_a, sigma2_s, sigma2_i, lam, B, bias, phi, tau_phi, lapse]

## bin_centers = make_bins(B, dx, binN)

In [6]:
#### make_bins (Theano version)
def make_bins(B, dx, binN):
    bins = (T.arange(binN)+1)*B
    bins = dx*bins/B

    tmp = T.scalar()
    
    bins = T.switch(T.eq(bins[-1],B),
                    T.set_subtensor(bins[-1], B+dx),
                    T.set_subtensor(bins[-1], 2*B - bins[-2]))
    
    bins = T.concatenate((-bins[::-1], T.zeros(1), bins))
    return bins

In [7]:
binN = T.cast(T.ceil(B/dx),'int32')
bin_centers = make_bins(B, dx, binN)

test_func = theano.function(
    inputs=[],
    outputs=[bin_centers, binN]
)

test_func()

[array([-4.2 , -4.  , -3.75, -3.5 , -3.25, -3.  , -2.75, -2.5 , -2.25,
        -2.  , -1.75, -1.5 , -1.25, -1.  , -0.75, -0.5 , -0.25,  0.  ,
         0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,  2.25,
         2.5 ,  2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.  ,  4.2 ]),
 array(17, dtype=int32)]

## F = Fmatrix([sigma, lambda, c], bin_centers)

In [11]:
def inner_loop(sbin, p, hp, lp, F, bin_centers,j):
    dd = bin_centers[hp] - bin_centers[lp]

    FF = T.set_subtensor(F[hp,j], F[hp,j]+p*(sbin-bin_centers[lp])/dd)
    FF = T.set_subtensor(FF[lp,j], FF[lp,j]+p*(bin_centers[hp]-sbin)/dd)

    F_rest = T.switch(T.eq(dd,0),
                      T.set_subtensor(F[lp,j], F[lp,j]+p),
                      FF)

    F = T.switch(T.le(sbin,bin_centers[0]),
                      T.set_subtensor(F[0,j], F[0,j]+p),
                      F)
    F = T.switch(T.ge(sbin,bin_centers[-1]),
                      T.set_subtensor(F[-1,j], F[-1,j]+p),
                      F)
    F = T.switch(T.gt(sbin,bin_centers[0]) & T.lt(sbin,bin_centers[-1]),
                      F_rest,
                      F)


    return F

In [22]:
def outer_loop(mu,j, F, base_sbins, ps, bin_centers):
    sbins = base_sbins + mu

    n = bin_centers.shape[0]-1

    hps = T.cast(T.ceil( (sbins-bin_centers[1])/dx) +1,'int32')
    lps = T.cast(T.floor((sbins-bin_centers[1])/dx) +1,'int32')
    
    hps = T.set_subtensor(hps[(hps < 0).nonzero()],0)
    lps = T.set_subtensor(lps[(lps < 0).nonzero()],0)

    hps = T.set_subtensor(hps[(hps > n).nonzero()],n)
    lps = T.set_subtensor(lps[(lps > n).nonzero()],n)
    
    hps = T.set_subtensor(hps[T.and_(bin_centers[0]<sbins, sbins<bin_centers[1]).nonzero()],1)
    lps = T.set_subtensor(lps[T.and_(bin_centers[0]<sbins, sbins<bin_centers[1]).nonzero()],0)
    
    hps = T.set_subtensor(hps[T.and_(bin_centers[-2]<sbins, sbins<bin_centers[-1]).nonzero()],n)
    lps = T.set_subtensor(lps[T.and_(bin_centers[-2]<sbins, sbins<bin_centers[-1]).nonzero()],n-1)

    hps = T.set_subtensor(hps[(bin_centers[-1]<sbins).nonzero()],n)
    lps = T.set_subtensor(lps[(bin_centers[-1]<sbins).nonzero()],n)

    hps = T.set_subtensor(hps[(sbins<bin_centers[0]).nonzero()],0)
    lps = T.set_subtensor(lps[(sbins<bin_centers[0]).nonzero()],0)

    # sequences: sbins, ps, hps, lps // 
    # prior results: F //
    # non-sequences: bin_centers, index j // 
    results,_ = theano.scan(fn=inner_loop,
                           outputs_info = F,
                           sequences = [sbins, ps, hps, lps],
                           non_sequences = [bin_centers,j]
                           )
    F = results[-1]

    return F

In [13]:
#### Fmatrix (Theano version)
def Fmatrix_i(params, bin_centers):
    global dt,dx
    sigma2 = params[0]
    lam = params[1]
    c = params[2]
    
    sigma2_sbin = sigma2
    
    F = T.zeros((bin_centers.shape[0],bin_centers.shape[0]),'float32')
    
    # for Fi (when lambda = 0)
    mus = bin_centers*T.exp(lam*dt)

#     mus = T.exp(lam*dt)*(bin_centers + c/lam) - c/lam
        
    tmp = T.ceil(10*T.sqrt(sigma2_sbin)/dx)
    n_sbins = T.switch(T.ge(tmp,70),tmp,70)
    
    swidth = 5*T.sqrt(sigma2_sbin)
    sbins = (T.arange(n_sbins)+1)/n_sbins*swidth#np.linspace(-swidth,swidth,n_sbins*2+1)
    sbins = T.concatenate((-sbins[::-1], T.zeros(1), sbins))

    ps = T.exp(-sbins**2/(2*sigma2)) 
    ps = ps/T.sum(ps)
            
    base_sbins = sbins
        
    # sequences: mus[1:], array(1:binsize) // mus, indices of the outer loop
    # prior results: zeros(size(F)) // initialize F as zeros
    # non-sequences: base_bins, ps, bin_centers // 
    res,_ = theano.scan(fn = outer_loop,
                      outputs_info = T.zeros_like(F),
                      sequences = [mus[1:], T.arange(bin_centers.shape[0]-1)+1],
                      non_sequences = [base_sbins, ps, bin_centers]
                     )
    F = res[-1]
    
    F = T.set_subtensor(F[:,0], 0)
    F = T.set_subtensor(F[:,-1], 0)
    F = T.set_subtensor(F[0,0], 1)
    F = T.set_subtensor(F[-1,-1], 1)
    
    return F

In [14]:
#### Fmatrix (Theano version)
def Fmatrix(params, bin_centers):
    global dt,dx
    sigma2 = params[0]
    lam = params[1]
    c = params[2]
    
    sigma2_sbin = sigma2
    
    F = T.zeros((bin_centers.shape[0],bin_centers.shape[0]),'float32')
    
#     mus = T.switch(T.eq(lam,0),bin_centers*T.exp(lam*dt),T.exp(lam*dt)*(bin_centers + c/lam) - c/lam)
    mus = T.exp(lam*dt)*(bin_centers + c/lam) - c/lam
        
    tmp = T.ceil(10*T.sqrt(sigma2_sbin)/dx)
    n_sbins = T.switch(T.ge(tmp,70),tmp,70)
    
    swidth = 5*T.sqrt(sigma2_sbin)
    sbins = (T.arange(n_sbins)+1)/n_sbins*swidth#np.linspace(-swidth,swidth,n_sbins*2+1)
    sbins = T.concatenate((-sbins[::-1], T.zeros(1), sbins))

    ps = T.exp(-sbins**2/(2*sigma2)) 
    ps = ps/T.sum(ps)
            
    base_sbins = sbins
        
        
    # sequences: mus[1:], array(1:binsize) // mus, indices of the outer loop
    # prior results: zeros(size(F)) // initialize F as zeros
    # non-sequences: base_bins, ps, bin_centers // 
    res,_ = theano.scan(fn = outer_loop,
                      outputs_info = T.zeros_like(F),
                      sequences = [mus[1:], T.arange(bin_centers.shape[0]-1)+1],
                      non_sequences = [base_sbins, ps, bin_centers]
                     )
    F = res[-1]
    
    F = T.set_subtensor(F[:,0], 0)
    F = T.set_subtensor(F[:,-1], 0)
    F = T.set_subtensor(F[0,0], 1)
    F = T.set_subtensor(F[-1,-1], 1)
    
    return F

In [17]:
Fi= Fmatrix_i([sigma2_i, 0, 0.0], bin_centers)
 

test_func = theano.function(
    inputs=[],
    outputs=Fi
)

test_func()

array([[ 1.        ,  0.41181999,  0.218907  , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.19714171,  0.1721313 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.18722765,  0.21792339, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.21792339,
         0.18722765,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.1721313 ,
         0.19714172,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.21890698,
         0.41181993,  1.        ]], dtype=float32)

In [19]:
F= Fmatrix([1.0000 ,0.1, 0.2], bin_centers)
 

test_func = theano.function(
    inputs=[],
    outputs=F
)

test_func()

array([[ 1.        ,  0.46112511,  0.36500674, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.09084516,  0.08595917, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.09459484,  0.10083522, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.09992243,
         0.09525968,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.0863713 ,
         0.09014276,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.36821347,
         0.46453354,  1.        ]], dtype=float32)

## LL = logProbRight(params)

params = [sigma_a, sigma_s, sigma_i, lambda, B, bias, phi, tau_phi, lapse]

In [57]:
def logLike(params, RightClickTimes, LeftClickTimes, Time_bins, rat_choice):
    LP_right = logProbRight(params, RightClickTimes, LeftClickTimes, Time_bins)

    return T.switch(T.gt(rat_choice,0),
                    LP_right,
                    T.log(1-T.exp(LP_right)))
    
def logProbRight(params, RightClickTimes, LeftClickTimes, Time_bins):
    """
    :params : arrays of parameters
    : sigma2_a
    : sigma2_s
    : sigma2_i
    : lam 
    : B
    : bias
    : phi
    : tau_phi
    : lapse
    """

    sigma2_a = params[0]
    sigma2_s = params[1]
    sigma2_i = params[2]
    lam = params[3]
    B = params[4]
    bias = params[5]
    phi = params[6]
    tau_phi = params[7]
    lapse = params[8]
    
    pright = T.dscalar()
    
        
    # ==== inter-click-intervals and Make adapted clicks
    
    net_input = T.vector("net_input")
    total_input = T.vector("total_input")


    # :: can be integrated with one function
    
    # sequences: T.extra_ops.diff(LeftClickTimes) // inter-click-intervals
    # prior results: lsame (0) // initial value
    # non-sequences: phi, tau_phi // parameters
    
    Lsame, updates = theano.scan(lambda ici, lsame, phi, tau_phi :
                                     1-T.exp((tau_phi*T.log(1-lsame*phi)-ici)/tau_phi),
                                  outputs_info = T.zeros(1),
                                  sequences=[T.extra_ops.diff(LeftClickTimes)],
                                  non_sequences=[phi, tau_phi]
                                  )
    Rsame, updates = theano.scan(lambda ici, rsame, phi, tau_phi : 
                                 1-T.exp((tau_phi*T.log(1-rsame*phi)-ici)/tau_phi),
                                  outputs_info = T.zeros(1),
                                  sequences=[T.extra_ops.diff(RightClickTimes)],
                                  non_sequences=[phi, tau_phi]
                                  )

    Lsame = T.concatenate([T.zeros(1, dtype=Lsame.dtype), Lsame[:,0]],axis=0)
    Rsame = T.concatenate([T.zeros(1, dtype=Rsame.dtype), Rsame[:,0]],axis=0)

    L_here =  T.cast(T.floor((LeftClickTimes+epsilon)/dt),'int32')
    R_here =  T.cast(T.floor((RightClickTimes+epsilon)/dt),'int32')

    dtype=theano.config.floatX

    template = T.vector('template')

    ## ==== Collect the adapted click values
    # index starts from 0
    # net_input / total_input

    # sequences: T.floor((LeftClickTimes+epsilon)/dt) // location
    # prior results: zeros // initial value
    # non-sequences: Lsame // parameters
    net_input_l, updates = theano.scan(lambda lhere,lsame, tmp :
                                     T.set_subtensor(tmp[lhere],tmp[lhere]+lsame),
                                  outputs_info = T.zeros_like(Time_bins),
                                  sequences = [L_here, Lsame]
                                  )
    net_input_r, updates = theano.scan(lambda rhere,rsame, tmp :
                                     T.set_subtensor(tmp[rhere],tmp[rhere]+rsame),
                                  outputs_info = T.zeros_like(Time_bins),
                                  sequences = [R_here, Rsame]
                                  )

    net_input = net_input_r[-1] - net_input_l[-1]
    total_input = net_input_r[-1] + net_input_l[-1]
    
    ## ==== make bins
    binN = T.cast(T.ceil(B/dx),'int32')
    bin_centers = make_bins(B,dx,binN)
        
    ## ==== make P init
    a0 = T.zeros_like(bin_centers)
    a0 = T.set_subtensor(a0[binN],1-2*lapse)
    a0 = T.set_subtensor(a0[0],lapse)
    a0 = T.set_subtensor(a0[-1],lapse)
    
    ## ==== Fi
    Fi = Fmatrix_i([sigma2_i, 0, 0.0], bin_centers)
    
    a = T.dot(Fi,a0)
 
    F0 = Fmatrix([sigma2_a*dt, lam, 0.0], bin_centers)
    
    
    def a_tracing(tot_input, net_input, a, F0, bin_centers, sigma2_a, sigma2_s, lam):
        total_var = sigma2_a*dt + (sigma2_s*tot_input)/40
        F = Fmatrix([total_var, lam, net_input/dt], bin_centers)
        a_rest = T.dot(F,a)
        
        a = T.switch(T.eq(tot_input,0),
                     T.dot(F0,a),
                     a_rest
                     )
        
        return a
        
    # sequences: T.floor((LeftClickTimes+epsilon)/dt) // location
    # prior results: zeros // initial value
    # non-sequences: Lsame // parameters
    res, _ = theano.scan(fn = a_tracing,
                             outputs_info = a,
                             sequences = [total_input[:-1], net_input[:-1]],
                             non_sequences = [F0,bin_centers,sigma2_a,sigma2_s,lam]
                        )

    a = res[-1]
    
    bias_hp = T.cast(T.ceil((bias-bin_centers[1])/dx) + 1,'int32') # top
    bias_lp = T.cast(T.floor((bias-bin_centers[1])/dx) + 1,'int32')
    
    Pd = T.zeros_like(bin_centers)
    Pd = T.set_subtensor(Pd[-bias_hp+1:],a[-bias_hp+1:])
    
    dh = bin_centers[bias_hp] - bias
    dl = bias - bin_centers[bias_lp]
    dd = dh + dl
    
    Pd_rest = T.set_subtensor(Pd[bias_hp],a[bias_hp]*(0.5+dh/dd/2))
    Pd_rest = T.set_subtensor(Pd_rest[bias_lp],a[bias_lp]*(dh/dd/2))
    
    Pd = T.switch(T.eq(bias_hp,bias_lp),
                  T.set_subtensor(Pd[bias_lp],a[bias_lp]/2),
                 Pd_rest)
    
    pright = T.sum(Pd)
    
    return T.log(pright)
    
# =========== Declare Theano symbolic variables =========== #
## inputs
RightClickTimes = T.dvector("RightClickTimes") # Right Clicks
LeftClickTimes = T.dvector("LeftClickTimes") # Left Clicks
Time_bins = T.vector("Time_bins") # Time_bins
rat_choice = T.wscalar("rat_choice") # rat_choice

# ==================== Construct Model ========================= #

LL = logLike(params, RightClickTimes, LeftClickTimes, Time_bins, rat_choice)
# grads = T.grad(LL,bias)
grads = []
for param in params:
    grad = T.grad(LL, param)
    grads.append(grad)

out_grads = T.stack(*grads)
    
l_rate = 0.1
updates = []
for param, grads in zip(params,grads):
    updates.append((param, param - grad * l_rate))

practice_train = theano.function(
    inputs = [RightClickTimes, LeftClickTimes, Time_bins, rat_choice],
    outputs = [LL, out_grads]
)

    
total_LL = 0.

# Training cycle
for i in range(3):
    RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)
    np_Nstep = int(np.ceil(maxT1/dt))
    np_empty_tmp = np.zeros((np_Nstep), dtype=None)

    [LLi, gradi] = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp, rat_choice1)

    total_LL += LLi

    print str(i+1)+ " - LL : " + str(LLi) + ", grad : " + str(gradi)


#     print "Trial :", '%04d' % (i+1), "LL = ", "{:.9f}".format(total_LL)

print "Done."


1 - LL : -2.53525610612, grad : [         nan          nan          nan          nan          nan
   0.98222984          nan          nan  11.7996249 ]
2 - LL : -0.240068572614, grad : [        nan         nan         nan         nan         nan -0.35468174
         nan         nan -0.8096263 ]
3 - LL : -0.0531115731168, grad : [        nan         nan         nan         nan         nan -0.00750378
         nan         nan -1.050503  ]
Done.


In [55]:
grads = T.grad(LL,params)
out_grads = T.stack(*grads)

# -> correct
# grads = []
# for param in params:
#     grad = T.grad(LL, param)
#     grads.append(grad)

# -> correct
# grads = [T.grad(LL,param) for param in params]

# l_rate = 0.1
# updates = []
# for param, grads in zip(params,grads):
#     updates.append((param, param - grad * l_rate))

practice_train = theano.function(
    inputs = [RightClickTimes, LeftClickTimes, Time_bins, rat_choice],
    outputs = [LL, out_grads]
)

    
total_LL = 0.

# Training cycle
for i in range(27):
    RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)
    np_Nstep = int(np.ceil(maxT1/dt))
    np_empty_tmp = np.zeros((np_Nstep), dtype=None)

    [LLi, gradi] = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp, rat_choice1)

    total_LL += LLi

    print(str(i+1)+ " - LL : " + str(LLi) + ", grad : " + str(gradi[0]) + ', ' + str(gradi[1]) + ', '+ str(gradi[2]) 
+ ', '+ str(gradi[3]) + ', '+ str(gradi[4]) + ', '+ str(gradi[5]) + ', '+ str(gradi[6]) + ', '
+ str(gradi[7]) + ', '+ str(gradi[8]))



#     print "Trial :", '%04d' % (i+1), "LL = ", "{:.9f}".format(total_LL)

print "Done."



1 - LL : -2.53525610612, grad : nan, nan, nan, nan, nan, 0.982229838835, nan, nan, 11.7996248966
2 - LL : -0.240068572614, grad : nan, nan, nan, nan, nan, -0.354681742035, nan, nan, -0.809626304237
3 - LL : -0.0531115731168, grad : nan, nan, nan, nan, nan, -0.00750377653089, nan, nan, -1.05050300257
4 - LL : -2.83579155537, grad : nan, nan, nan, nan, nan, -0.489470602281, nan, nan, 16.7154290466
5 - LL : -0.186951986201, grad : nan, nan, nan, nan, nan, 0.322187832741, nan, nan, -0.882700667007
6 - LL : -1.20052273434, grad : nan, nan, nan, nan, nan, -1.23899377678, nan, nan, 1.46872546171
7 - LL : -0.0512972358982, grad : nan, nan, nan, nan, nan, 1.57516917233e-05, nan, nan, -1.05262696898
8 - LL : -0.0513274642198, grad : nan, nan, nan, nan, nan, -0.000151265836819, nan, nan, -1.05259161355
9 - LL : -1.4896305146, grad : nan, nan, nan, nan, nan, 1.63599930791, nan, nan, 2.70606264428
10 - LL : -0.0713735568108, grad : nan, nan, nan, nan, nan, -0.0546409493745, nan, nan, -1.02890850626

In [23]:

total_LL = 0.

# Training cycle
for i in range(27):
    RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)
    np_Nstep = int(np.ceil(maxT1/dt))
    np_empty_tmp = np.zeros((np_Nstep), dtype=None)

    LLi = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp, rat_choice1)

    total_LL += LLi

    print(str(i+1)+ " - LL : " + str(LLi))

print "Done."



1 - LL : -2.53525610612
2 - LL : -0.240068572614
3 - LL : -0.0531115731168
4 - LL : -2.83579155537
5 - LL : -0.186951986201
6 - LL : -1.20052273434
7 - LL : -0.0512972358982
8 - LL : -0.0513274642198
9 - LL : -1.4896305146
10 - LL : -0.0713735568108
11 - LL : -0.979718758989
12 - LL : -0.0515012016706
13 - LL : -0.244442007701
14 - LL : -2.94919279661
15 - LL : -0.542772630183
16 - LL : -0.0513611903175
17 - LL : -0.0532301545946
18 - LL : -1.01690648472
19 - LL : -0.0520756615757
20 - LL : -0.162860320651
21 - LL : -0.0532741293723
22 - LL : -2.99530324698
23 - LL : -0.797323944961
24 - LL : -1.39407686843
25 - LL : -0.0671278847861
26 - LL : -0.0512951210377
27 - LL : -0.0945154790448
Done.



## Backwards

In [ ]:
def logLike(params, RightClickTimes, LeftClickTimes, Time_bins, rat_choice):
    LP_right = logProbRight(params, RightClickTimes, LeftClickTimes, Time_bins)

    return T.switch(T.gt(rat_choice,0),
                    LP_right,
                    T.log(1-T.exp(LP_right)))
    
def logProbRight(params, RightClickTimes, LeftClickTimes, Time_bins):
    """
    :params : arrays of parameters
    : sigma2_a
    : sigma2_s
    : sigma2_i
    : lam 
    : B
    : bias
    : phi
    : tau_phi
    : lapse
    """

    sigma2_a = params[0]
    sigma2_s = params[1]
    sigma2_i = params[2]
    lam = params[3]
    B = params[4]
    bias = params[5]
    phi = params[6]
    tau_phi = params[7]
    lapse = params[8]
    
    pright = T.dscalar()
    
        
    # ==== inter-click-intervals and Make adapted clicks
    
    net_input = T.vector("net_input")
    total_input = T.vector("total_input")


    # :: can be integrated with one function
    
    # sequences: T.extra_ops.diff(LeftClickTimes) // inter-click-intervals
    # prior results: lsame (0) // initial value
    # non-sequences: phi, tau_phi // parameters
    
    Lsame, updates = theano.scan(lambda ici, lsame, phi, tau_phi :
                                     1-T.exp((tau_phi*T.log(1-lsame*phi)-ici)/tau_phi),
                                  outputs_info = T.zeros(1),
                                  sequences=[T.extra_ops.diff(LeftClickTimes)],
                                  non_sequences=[phi, tau_phi]
                                  )
    Rsame, updates = theano.scan(lambda ici, rsame, phi, tau_phi : 
                                 1-T.exp((tau_phi*T.log(1-rsame*phi)-ici)/tau_phi),
                                  outputs_info = T.zeros(1),
                                  sequences=[T.extra_ops.diff(RightClickTimes)],
                                  non_sequences=[phi, tau_phi]
                                  )

    Lsame = T.concatenate([T.zeros(1, dtype=Lsame.dtype), Lsame[:,0]],axis=0)
    Rsame = T.concatenate([T.zeros(1, dtype=Rsame.dtype), Rsame[:,0]],axis=0)

    L_here =  T.cast(T.floor((LeftClickTimes+epsilon)/dt),'int32')
    R_here =  T.cast(T.floor((RightClickTimes+epsilon)/dt),'int32')

    dtype=theano.config.floatX

    template = T.vector('template')

    ## ==== Collect the adapted click values
    # index starts from 0
    # net_input / total_input

    # sequences: T.floor((LeftClickTimes+epsilon)/dt) // location
    # prior results: zeros // initial value
    # non-sequences: Lsame // parameters
    net_input_l, updates = theano.scan(lambda lhere,lsame, tmp :
                                     T.set_subtensor(tmp[lhere],tmp[lhere]+lsame),
                                  outputs_info = T.zeros_like(Time_bins),
                                  sequences = [L_here, Lsame]
                                  )
    net_input_r, updates = theano.scan(lambda rhere,rsame, tmp :
                                     T.set_subtensor(tmp[rhere],tmp[rhere]+rsame),
                                  outputs_info = T.zeros_like(Time_bins),
                                  sequences = [R_here, Rsame]
                                  )

    net_input = net_input_r[-1] - net_input_l[-1]
    total_input = net_input_r[-1] + net_input_l[-1]
    
    ## ==== make bins
    binN = T.cast(T.ceil(B/dx),'int32')
    bin_centers = make_bins(B,dx,binN)
        
    ## ==== make P init
    a0 = T.zeros_like(bin_centers)
    a0 = T.set_subtensor(a0[binN],1-2*lapse)
    a0 = T.set_subtensor(a0[0],lapse)
    a0 = T.set_subtensor(a0[-1],lapse)
    
    ## ==== Fi
    Fi = Fmatrix_i([sigma2_i, 0, 0.0], bin_centers)
    
    a = T.dot(Fi,a0)
 
    F0 = Fmatrix([sigma2_a*dt, lam, 0.0], bin_centers)
    
    
    def a_tracing(tot_input, net_input, a, F0, bin_centers, sigma2_a, sigma2_s, lam):
        total_var = sigma2_a*dt + (sigma2_s*tot_input)/40
        F = Fmatrix([total_var, lam, net_input/dt], bin_centers)
        a_rest = T.dot(F,a)
        
        a = T.switch(T.eq(tot_input,0),
                     T.dot(F0,a),
                     a_rest
                     )
        
        return a
        
    # sequences: T.floor((LeftClickTimes+epsilon)/dt) // location
    # prior results: zeros // initial value
    # non-sequences: Lsame // parameters
    res, _ = theano.scan(fn = a_tracing,
                             outputs_info = a,
                             sequences = [total_input[:-1], net_input[:-1]],
                             non_sequences = [F0,bin_centers,sigma2_a,sigma2_s,lam]
                        )

    a = res[-1]
    a_trace = res
    
    bias_hp = T.cast(T.ceil((bias-bin_centers[1])/dx) + 1,'int32') # top
    bias_lp = T.cast(T.floor((bias-bin_centers[1])/dx) + 1,'int32')
    
    Pd = T.zeros_like(bin_centers)
    Pd = T.set_subtensor(Pd[-bias_hp+1:],a[-bias_hp+1:])
    
    dh = bin_centers[bias_hp] - bias
    dl = bias - bin_centers[bias_lp]
    dd = dh + dl
    
    Pd_rest = T.set_subtensor(Pd[bias_hp],a[bias_hp]*(0.5+dh/dd/2))
    Pd_rest = T.set_subtensor(Pd_rest[bias_lp],a[bias_lp]*(dh/dd/2))
    
    Pd = T.switch(T.eq(bias_hp,bias_lp),
                  T.set_subtensor(Pd[bias_lp],a[bias_lp]/2),
                 Pd_rest)
    
    pright = T.sum(Pd)
    
    return T.log(pright)
    
# =========== Declare Theano symbolic variables =========== #
## inputs
RightClickTimes = T.dvector("RightClickTimes") # Right Clicks
LeftClickTimes = T.dvector("LeftClickTimes") # Left Clicks
Time_bins = T.vector("Time_bins") # Time_bins
rat_choice = T.wscalar("rat_choice") # rat_choice

# ==================== Construct Model ========================= #

LL = logLike(params, RightClickTimes, LeftClickTimes, Time_bins, rat_choice)

# grads = []
# for param in params:
#     grad = T.grad(LL, param)
#     grads.append(grad)

# out_grads = T.stack(*grads)
    
# l_rate = 0.1
# updates = []
# for param, grads in zip(params,grads):
#     updates.append((param, param - grad * l_rate))

practice_train = theano.function(
    inputs = [RightClickTimes, LeftClickTimes, Time_bins, rat_choice],
    outputs = LL#[LL, out_grads]
)

    
total_LL = 0.

# Training cycle
for i in range(3):
    RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)
    np_Nstep = int(np.ceil(maxT1/dt))
    np_empty_tmp = np.zeros((np_Nstep), dtype=None)

    LLi = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp, rat_choice1)

    total_LL += LLi

    print str(i+1)+ " - LL : " + str(LLi)
#     print str(i+1)+ " - LL : " + str(LLi) + ", grad : " + str(gradi)


#     print "Trial :", '%04d' % (i+1), "LL = ", "{:.9f}".format(total_LL)

print "Done."

In [198]:
def test_logProbRight(params, RightClickTimes, LeftClickTimes, Time_bins):
    """
    :params : arrays of parameters
    : sigma2_a
    : sigma2_s
    : sigma2_i
    : lam 
    : B
    : bias
    : phi
    : tau_phi
    : lapse
    """

    sigma2_a = params[0]
    sigma2_s = params[1]
    sigma2_i = params[2]
    lam = params[3]
    B = params[4]
    bias = params[5]
    phi = params[6]
    tau_phi = params[7]
    lapse = params[8]
    
    pright = T.dscalar()
    
        
    # ==== inter-click-intervals and Make adapted clicks
    
    net_input = T.vector("net_input")
    total_input = T.vector("total_input")


    # :: can be integrated with one function
    
    # sequences: T.extra_ops.diff(LeftClickTimes) // inter-click-intervals
    # prior results: lsame (0) // initial value
    # non-sequences: phi, tau_phi // parameters
    
    Lsame, updates = theano.scan(lambda ici, lsame, phi, tau_phi :
                                     1-T.exp((tau_phi*T.log(1-lsame*phi)-ici)/tau_phi),
                                  outputs_info = T.zeros(1),
                                  sequences=[T.extra_ops.diff(LeftClickTimes)],
                                  non_sequences=[phi, tau_phi]
                                  )
    Rsame, updates = theano.scan(lambda ici, rsame, phi, tau_phi : 
                                 1-T.exp((tau_phi*T.log(1-rsame*phi)-ici)/tau_phi),
                                  outputs_info = T.zeros(1),
                                  sequences=[T.extra_ops.diff(RightClickTimes)],
                                  non_sequences=[phi, tau_phi]
                                  )

    Lsame = T.concatenate([T.zeros(1, dtype=Lsame.dtype), Lsame[:,0]],axis=0)
    Rsame = T.concatenate([T.zeros(1, dtype=Rsame.dtype), Rsame[:,0]],axis=0)

    L_here =  T.cast(T.floor((LeftClickTimes+epsilon)/dt),'int32')
    R_here =  T.cast(T.floor((RightClickTimes+epsilon)/dt),'int32')

    dtype=theano.config.floatX

    template = T.vector('template')

    ## ==== Collect the adapted click values
    # index starts from 0
    # net_input / total_input

    # sequences: T.floor((LeftClickTimes+epsilon)/dt) // location
    # prior results: zeros // initial value
    # non-sequences: Lsame // parameters
    net_input_l, updates = theano.scan(lambda lhere,lsame, tmp :
                                     T.set_subtensor(tmp[lhere],tmp[lhere]+lsame),
                                  outputs_info = T.zeros_like(Time_bins),
                                  sequences = [L_here, Lsame]
                                  )
    net_input_r, updates = theano.scan(lambda rhere,rsame, tmp :
                                     T.set_subtensor(tmp[rhere],tmp[rhere]+rsame),
                                  outputs_info = T.zeros_like(Time_bins),
                                  sequences = [R_here, Rsame]
                                  )

    net_input = net_input_r[-1] - net_input_l[-1]
    total_input = net_input_r[-1] + net_input_l[-1]
    
    ## ==== make bins
    binN = T.cast(T.ceil(B/dx),'int32')
    bin_centers = make_bins(B,dx,binN)
        
    ## ==== make P init
    a0 = T.zeros_like(bin_centers)
    a0 = T.set_subtensor(a0[binN],1-2*lapse)
    a0 = T.set_subtensor(a0[0],lapse)
    a0 = T.set_subtensor(a0[-1],lapse)
    
    ## ==== Fi
    Fi = Fmatrix_i([sigma2_i, 0, 0.0], bin_centers)
    
    a = T.dot(Fi,a0)
    ai = a
    
    F0 = Fmatrix([sigma2_a*dt, lam, 0.0], bin_centers)
    
    
    def a_tracing(tot_input, net_input, a, F0, bin_centers, sigma2_a, sigma2_s, lam):
        total_var = sigma2_a*dt + (sigma2_s*tot_input)/40
        F = Fmatrix([total_var, lam, net_input/dt], bin_centers)
        a_rest = T.dot(F,a)
        
        a = T.switch(T.eq(tot_input,0),
                     T.dot(F0,a),
                     a_rest
                     )
        F_res = T.switch(T.eq(tot_input,0),
                     F0,
                     F
                     )
        
        return a, F_res
        
    # sequences: T.floor((LeftClickTimes+epsilon)/dt) // location
    # prior results: zeros // initial value
    # non-sequences: Lsame // parameters
    [a_res, F_res], _ = theano.scan(fn = a_tracing,
                             outputs_info = [a, None],
                             sequences = [total_input[:-1], net_input[:-1]],
                             non_sequences = [F0,bin_centers,sigma2_a,sigma2_s,lam]
                        )

    a = a_res[-1]
    a_trace = T.concatenate([[ai],a_res],axis=0)
    
    bias_hp = T.cast(T.ceil((bias-bin_centers[1])/dx) + 1,'int32') # top
    bias_lp = T.cast(T.floor((bias-bin_centers[1])/dx) + 1,'int32')
    
    Pd = T.zeros_like(bin_centers)
    Pd = T.set_subtensor(Pd[-bias_hp+1:],a[-bias_hp+1:])
    
    dh = bin_centers[bias_hp] - bias
    dl = bias - bin_centers[bias_lp]
    dd = dh + dl
    
    Pd_rest = T.set_subtensor(Pd[bias_hp],a[bias_hp]*(0.5+dh/dd/2))
    Pd_rest = T.set_subtensor(Pd_rest[bias_lp],a[bias_lp]*(dh/dd/2))
    
    Pd = T.switch(T.eq(bias_hp,bias_lp),
                  T.set_subtensor(Pd[bias_lp],a[bias_lp]/2),
                 Pd_rest)
    
    pright = T.sum(Pd)
    
#     return T.log(pright)
    return Pd
    
# =========== Declare Theano symbolic variables =========== #
## inputs
RightClickTimes = T.dvector("RightClickTimes") # Right Clicks
LeftClickTimes = T.dvector("LeftClickTimes") # Left Clicks
Time_bins = T.vector("Time_bins") # Time_bins
rat_choice = T.wscalar("rat_choice") # rat_choice

# ==================== Construct Model ========================= #

test_mat = test_logProbRight(params, RightClickTimes, LeftClickTimes, Time_bins)

# grads = []
# for param in params:
#     grad = T.grad(LL, param)
#     grads.append(grad)

# out_grads = T.stack(*grads)
    
# l_rate = 0.1
# updates = []
# for param, grads in zip(params,grads):
#     updates.append((param, param - grad * l_rate))

practice_train = theano.function(
    inputs = [RightClickTimes, LeftClickTimes, Time_bins],
    outputs = test_mat#[LL, out_grads]
)

    
total_LL = 0.

# Training cycle
# for i in range(3):
i = 0
RClickTimes1, LClickTimes1, maxT1, rat_choice1 = trialdata(ratdata, i)
np_Nstep = int(np.ceil(maxT1/dt))
np_empty_tmp = np.zeros((np_Nstep), dtype=None)

out = practice_train(RClickTimes1, LClickTimes1, np_empty_tmp)

#     total_LL += LLi

print out
#     print str(i+1)+ " - LL : " + str(LLi) + ", grad : " + str(gradi)


[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.00430198  0.01966138  0.03868644
  0.05593398  0.07428353  0.09061925  0.10154686  0.10452836  0.09883798
  0.08584866  0.06849483  0.05019817  0.0337917   0.02089319  0.01186436
  0.00618718  0.00278642  0.05229431]


In [86]:
np.shape(out)

(17, 35, 35)

In [82]:
np.sum(out,1)

array([ 1.        ,  1.00000004,  1.00000007,  1.00000009,  1.00000011,
        1.00000014,  1.00000018,  1.00000018,  1.00000018,  1.00000018,
        1.00000019,  1.00000016,  1.00000016,  1.00000017,  1.00000017,
        1.00000018,  1.00000019,  1.00000018])

In [83]:
a_trace = out

In [87]:
Fs = out

In [98]:
(Fs[::-1][:][:])

array([[[  1.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.13291435e-05,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        ..., 
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           3.19388482e-06,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           9.99996841e-01,   1.00000000e+00,   1.00000000e+00]],

       [[  1.00000000e+00,   2.56767362e-01,   1.07986946e-02, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   5.26334405e-01,   2.05956832e-0

In [109]:
a_trace[-1:0:-1][:]

array([[  5.00000000e-02,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   9.99945496e-07,
          1.43592953e-05,   1.33464167e-04,   9.28853697e-04,
          4.78058327e-03,   1.82021007e-02,   5.14239348e-02,
          1.07945301e-01,   1.68504886e-01,   1.96131049e-01,
          1.68504886e-01,   1.07945294e-01,   5.14239281e-02,
          1.82021024e-02,   4.78058285e-03,   9.28853592e-04,
          1.33464180e-04,   1.43592937e-05,   9.99945496e-07,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   5.00000000e-02],
       [  5.00000000e-02,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.96932328e-15,
          9.70609611e-11,   1.88652547e-08,   5.41821577e-07,
          6.96280487e-06,   6.18292926e-05,   4.18950566e-04,
          2.18211144e-03,

In [127]:
np.shape(a_trace[-1:0:-1][:])

(17, 35)

In [129]:
np.shape(Fs[0][:][:])

(35, 35)

In [197]:
Tvec = T.vector()
Tvec2 = T.vector()
Tmat = T.matrix()

test_result = Tmat*Tvec
test_result = T.transpose(test_result)/Tvec2

test_func = theano.function(
    inputs = [Tmat, Tvec, Tvec2],
    outputs = test_result
)

print test_func(5*np.ones((5,5)), np.arange(5)+1,np.arange(5)+6)

[[ 0.83333333  0.71428571  0.625       0.55555556  0.5       ]
 [ 1.66666667  1.42857143  1.25        1.11111111  1.        ]
 [ 2.5         2.14285714  1.875       1.66666667  1.5       ]
 [ 3.33333333  2.85714286  2.5         2.22222222  2.        ]
 [ 4.16666667  3.57142857  3.125       2.77777778  2.5       ]]


In [199]:
Pd = out

In [286]:
def Bmatrix(F,Fdistribm1,Fdistrib):
    B = F*Fdistribm1
    B = T.transpose(B)/Fdistrib
    
    return B

def backwards(Pf, Pd, Fmats):#, net_input):
    # Pf 35x18
    # Fs 17x35x35
    
    # normalize to use as initial distribution of the backwards, posterior run
    Pd = Pd/T.sum(Pd)
    
    # the backwards, posterior probability distribution
    Pb = T.zeros_like(Pf)
    # last timestep of backwards distribution is Pd
    Pb = T.set_subtensor(Pb[:,-1],Pd)
    
    input_Fs = Fmats[::-1][:][:] # reversed sequence k-1 -> 1
    input_f_m1 = Pf[-2::-1] # reversed sequence k-1 -> 1
    input_f = Pf[-1:0:-1] # reversed sequence k -> 2
    
    # sequences: Fs, forward probability(k-1), forward probability(k-1) 
    # results: Bmatrix // initial value = zeros
    B,_ = theano.scan(fn = Bmatrix,
                    outputs_info = None,
                    sequences = [input_Fs, input_f_m1, input_f]                    
                   )
    
    return B

In [287]:
Tmat = T.matrix()
Tvec = T.vector()
Tmat2 = T.tensor3()

Bmat = backwards(Tmat, Tvec, Tmat2)

backwards_test = theano.function(
    inputs = [Tmat, Tvec, Tmat2],
    outputs = Bmat
)

print backwards_test(a_trace, Pd, Fs)

<TensorType(float64, 3D)>
Shape.0
Shape.0


UnusedInputError: theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 1 is not part of the computational graph needed to compute the outputs: <TensorType(float64, vector)>.
To make this error into a warning, you can pass the parameter on_unused_input='warn' to theano.function. To disable it completely, use on_unused_input='ignore'.

In [234]:
np.shape(a_trace[::-1][:])

(18, 35)

In [235]:
np.arange(3)[-1:0:-1]

array([2, 1])

In [239]:
np.arange(3)[-2::-1]

array([1, 0])

In [248]:
A_v = np.array([[[3.,2.],[4.,2.],[5.,2.]],[[3.,2.],[4.,2.],[5.,2.]],\
                [[1.,2.],[2.,2.],[3.,2.]],[[1.,2.],[2.,2.],[3.,2.]]])

In [249]:
np.shape(A_v)

(4, 3, 2)

In [250]:
np.shape(Fs)

(17, 35, 35)

In [269]:
Tmat2

<TensorType(float64, 3D)>

In [330]:
def Bmatrix(F,Fdistribm1,Fdistrib):
    B = F*Fdistribm1
    B = T.transpose(B)/Fdistrib
    
    return B


def tensor3_test(Pf, Fmats):
    # Pf 35x18
    # Fs 17x35x35
    
#     # normalize to use as initial distribution of the backwards, posterior run
#     Pd = Pd/T.sum(Pd)
    
#     # the backwards, posterior probability distribution
#     Pb = T.zeros_like(Pf)
#     # last timestep of backwards distribution is Pd
#     Pb = T.set_subtensor(Pb[:,-1],Pd)

    Fmats = Fmats
    avec = Pf[:-1][:] # for reversed sequence k-1 -> 1
    bvec = Pf[1:][:] # for reversed sequence k -> 2

    res,_ = theano.scan(fn = Bmatrix,
                outputs_info = None,
                sequences = [Fmats, avec, bvec],
                go_backwards = True
               )

    return res

out_val = tensor3_test(Tmat, Tmat2)

test = theano.function(
    inputs = [Tmat, Tmat2],
    outputs = out_val
)

tt = test(a_trace, Fs)
print tt

[[[  1.00000000e+00              nan   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00              nan   1.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00              nan   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  ..., 
  [  0.00000000e+00              nan   0.00000000e+00 ...,   0.00000000e+00
     1.24842118e-07   2.08272800e-03]
  [  0.00000000e+00              nan   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   5.83800788e-04]
  [  0.00000000e+00              nan   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   9.56488939e-01]]

 [[  9.99999937e-01   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  5.03452467e-08   2.58912166e-01   1.86650129e-02 ...,   0.00000000e+00
     0.00000000e+00   0.00000000e+00]
  [  1.27323026e-08   6.09233732e-01   3.05369962e-01 ...,   0.00000000e+00
  